# IMDB Reviews Text Analysis Regression

In [2]:
import requests
from urllib.parse import urljoin
import math
import csv
import time
import sqlite3
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from nltk.corpus import stopwords
from math import log

from sklearn import linear_model
from sklearn import preprocessing
import numpy as np

import pandas as pd

from sklearn.feature_extraction import text 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


#sqlite_file = "C:\\Users\\kevin\\Desktop\\CS 175\\Project\\sqlite\\my_db.sqlite"
stopWords = set(stopwords.words('english'))
#conn = sqlite3.connect(sqlite_file)
#c = conn.cursor()

First, lets analyze the spread of the number of reviews across all movies.

In [ ]:
review_rows = c.execute("SELECT imdb_id FROM reviews").fetchall()

review_counts = {}
count = 0
for review in review_rows:
    count += 1
    if count > 0:
        imdb_id = review[0]
        if imdb_id in review_counts:
            review_counts[imdb_id] += 1
        else:
            review_counts[imdb_id] = 1
        if (count % 100 == 0):
            print(count)
            


In [ ]:
# Selecting all movies that have at least 1 review in the first three weeks

review_rows = c.execute("SELECT imdb_id FROM reviews").fetchall()

imdb_ids = []
reviews = []
three_weeks_count = []

for review in review_rows:
    count += 1
    if count > 0:
        imdb_id = review[0]
        if imdb_id in review_counts:
            review_counts[imdb_id] += 1
        else:
            review_counts[imdb_id] = 1
        if (count % 100 == 0):
            print(count)
            

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(review_totals, normalized_boxoffice, test_size=.25)

x_train = np.reshape(x_train, (-1,1))
x_test = np.reshape(x_test, (-1,1))
linear_regression(x_train, x_test, y_train, y_test)
random_forests(x_train, x_test, y_train, y_test)

In [ ]:
print("Number of movies counted: " + str(len(review_counts)))


review_totals = []
above_40 = 0
above_30 = 0
above_150 = 0
above_20 = []
for imdb_id, review_total in review_counts.items():
    review_totals.append(review_total)
    if review_total > 150:
        above_150 += 1
    if review_total > 40:
        above_40 += 1
    if review_total > 30:
        above_30 += 1
    if review_total > 20:
        above_20.append(imdb_id)

review_totals = numpy.array(sorted(review_totals))
print("Mean of values: " + str(numpy.mean(review_totals)))
print("Median of values: " + str(numpy.median(review_totals)))
print("STD of values: " + str(numpy.std(review_totals)))
print("Minimum of values: " + str(min(review_totals)))
print("Max of values: " + str(max(review_totals)))

print("Number of movies w/ more than 150 reviews: " + str(above_150))
print("Number of movies w/ more than 40 reviews: " + str(above_40))
print("Number of movies w/ more than 30 reviews: " + str(above_30))

#### First, we'll collect all the reviews into datasets.

In [ ]:
stmt = "SELECT imdb_id, len, reviews FROM reviews_three"
response = c.execute(stmt).fetchall()

reviews = []
box_offices = []
imdb_ids = []
for row in response:
    if row[1] > 0:
        imdb_ids.append(row[0])
        reviews.append(row[2])
        box = c.execute("SELECT box FROM movies WHERE imdb_id = '" + row[0] + "'").fetchall()[0][0]
        box_offices.append(log(float(box))/log(10))

lab_enc = preprocessing.LabelEncoder()
box_offices = lab_enc.fit_transform(box_offices)

After checking all the movies, only 3999 out of the about 5000 have reviews within the first three weeks. Next, like put our data in to the correct format for our linear regression.


In [3]:
def extract_split_features(train_data, test_data):
    
    x_train, x_test, y_train, y_test = train_test_split(train_data, test_data, test_size=.25)

    count_vect = CountVectorizer(stop_words = stopWords, min_df = 3)
    X_train_counts = count_vect.fit_transform(x_train)
    X_test_counts = count_vect.transform(x_test)

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    
    features = count_vect.get_feature_names()
    
    print(top_mean_feats(X_train_tfidf, features, top_n = 50))


    return (X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf, y_train, y_test)

In [4]:
# Get top tfidf values in a row and return them with their corresponding feature names
# Source: https://buhrmann.github.io/tfidf-analysis.html

def top_tfidf_feats(row, features, top_n=25):
    
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [5]:
def linear_regression(X_train, X_test, Y_train, Y_test):
    
    from sklearn import linear_model
    clf = linear_model.LinearRegression().fit(X_train, Y_train)
    predicted_LR = clf.predict(X_test)
    
    mse = np.mean((predicted_LR - Y_test)**2)
    variance = clf.score(X_test, Y_test)
    
    print("Linear Regression: ")
    print("MSE= " + str(mse))
    print("variance= " + str(variance))
    
    return predicted_LR
    
    

In [6]:
def random_forests(X_train, X_test, Y_train, Y_test):

    from sklearn.ensemble import RandomForestRegressor
    
    clf = RandomForestRegressor(n_estimators = 10, max_depth = 4, random_state = 0).fit(X_train, Y_train)
    
    predicted_RF = clf.predict(X_test)
    mse = np.mean((predicted_RF - Y_test)**2)
    variance = clf.score(X_test, Y_test)
    
    print("Random Forests: ")
    print("MSE= " + str(mse))
    print("variance= " + str(variance))
    
    return clf

In [7]:
def support_vector(X_train, X_test, Y_train, Y_test):
    
    from sklearn.svm import SVR
    
    clf = SVR(C=1.0, epsilon=0.2).fit(X_train, Y_train) 
    
    predicted_SVR = clf.predict(X_test)
    mse = np.mean((predicted_SVR - Y_test)**2)
    variance = clf.score(X_test, Y_test)
    
    print("Support Vector Regression: ")
    print("MSE= " + str(mse))
    print("variance= " + str(variance))

In [13]:
from collections import defaultdict
import sqlite3 as sql

conn = sql.connect("C:\\Users\\Justin\\CS175\\movie_meta.db")
conn2 = sql.connect("C:\\Users\\Justin\\CS175\\comment_set1.db")

result_dict = defaultdict(list)

c = conn.cursor()
c2 = conn2.cursor()

c2.execute("SELECT DISTINCT imdb_id FROM comments")
unique = c2.fetchall()
unique_list = [x[0] for x in unique]

for x in unique_list:
        if x == 'tconst' : continue
        c.execute("SELECT box_office FROM meta WHERE imdb_id = ?",(x,))
        box_office = c.fetchall()[0][0]
        result_dict[x].append(math.log(box_office,10))
        result_dict[x].append("")

c2.execute("SELECT imdb_id, body FROM comments")
results = c2.fetchall()

for imdb_id, body in results:

        try:
                if imdb_id == 'tconst': continue
                result_dict[imdb_id][1] += body

        except Exception as e:
                continue

reviews = []
box_offices = []
for x, y in result_dict.items():
        box_offices.append(y[0])
        reviews.append(y[1])



In [14]:
X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf, Y_train, Y_test = extract_split_features(reviews, box_offices)

         feature     tfidf
0          movie  0.035666
1           like  0.020121
2           film  0.015711
3            one  0.010292
4         movies  0.008378
5         people  0.007283
6          would  0.007203
7           http  0.006654
8           good  0.006637
9            com  0.006162
10         think  0.005847
11        really  0.005459
12           man  0.005300
13           www  0.005227
14         watch  0.004727
15          imdb  0.004304
16          best  0.004262
17          time  0.003818
18          2011  0.003301
19         great  0.003265
20         title  0.003222
21         black  0.003185
22         films  0.003165
23        batman  0.003131
24        horror  0.003108
25           see  0.003074
26         scene  0.002996
27          year  0.002928
28            10  0.002928
29          bond  0.002926
30          also  0.002824
31          dead  0.002793
32        pretty  0.002790
33         story  0.002776
34           war  0.002766
35          last  0.002710
3

### Word Analysis

Let train our learner from just using the count vectorizers.

In [15]:
predicted_LR = linear_regression(X_train_counts, X_test_counts, Y_train, Y_test)
clf_RF = random_forests(X_train_counts, X_test_counts, Y_train, Y_test)


Linear Regression: 
MSE= 690.033898349
variance= -340.611352488
Random Forests: 
MSE= 2.00917464463
variance= 0.00532876807907


In [16]:
predicted_LR = linear_regression(X_train_tfidf, X_test_tfidf, Y_train, Y_test)


Linear Regression: 
MSE= 3.16276687971
variance= -0.565773904687


In [17]:
clf_RF = random_forests(X_train_tfidf, X_test_tfidf, Y_train, Y_test)


Random Forests: 
MSE= 1.96846840742
variance= 0.0254809849233


We noticed in our analysis that review sentiment has very little effect on box office. Lets try to remove polarizing words, to test if that is just causing additional noise. We will add in positive and negative words we found in an online corpus.

In [ ]:
polarized = []
fp = open('positive-words.txt')

for i, line in enumerate(fp):
    if i > 34:
        polarized.append(line.strip())


fp = open('negative-words.txt')

for i, line in enumerate(fp):
    if i > 34:
        polarized.append(line.strip())

fp.close()
new_stopwords = text.ENGLISH_STOP_WORDS.union(polarized)


def extract_split_features_no_polarized(train_data, test_data):
    
    x_train, x_test, y_train, y_test = train_test_split(train_data, test_data, test_size=.25)

    count_vect = CountVectorizer(stop_words = new_stopwords, min_df = 3)
    X_train_counts = count_vect.fit_transform(x_train)
    X_test_counts = count_vect.transform(x_test)

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)

    return (X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf, y_train, y_test)


In [ ]:
X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf, Y_train, Y_test = extract_split_features_no_polarized(reviews, box_offices)
predicted_LR = linear_regression(X_train_tfidf, X_test_tfidf, Y_train, Y_test)
clf_RF = random_forests(X_train_tfidf, X_test_tfidf, Y_train, Y_test)


# Combining the Learners

In [ ]:

review_totals = []
normalized_budgets = []
normalized_boxoffice = []
reviews = []


with open('movie_metadata.csv', 'r', encoding='utf-8') as moviecsv:
    moviecsv = csv.DictReader(moviecsv)
    count = 0
    for line in moviecsv:
        budget = line["Budget"]
        imdb_id = line['tconst']
        movie_reviews = c.execute("SELECT len, reviews FROM reviews_three WHERE imdb_id ='" + imdb_id + "'").fetchall()
        length = movie_reviews[0][0]
        da_reviews = movie_reviews[0][1]
        if budget != "0" and  length > 0 and budget != "LOOKINTO":
            count += 1
            
            review_totals.append(length)
            reviews.append(da_reviews)
            normalized_budgets.append(math.log(int(budget), 10))
            
            box_office = line['BoxOffice'].split(".")[0]
            normalized_boxoffice.append(math.log(int(box_office), 10))
            
            print(count)
            print(imdb_id)
        


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(review_totals, normalized_boxoffice, test_size=.25)


predicted_LR_lengths = linear_regression(np.reshape(x_train, (-1,1)) , np.reshape(x_test, (-1,1)), y_train, y_test)
predicted_RF_lengths = random_forests(np.reshape(x_train, (-1,1)) , np.reshape(x_test, (-1,1)), y_train, y_test)

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import r2_score


x_budget_train, x_budget_test, x_total_train, x_total_test, y_bo_train, y_bo_test = train_test_split(normalized_budgets, review_totals, normalized_boxoffice, test_size=.25)



In [ ]:

# print(len(x_train_tfidf))
print(len(x_budget_train))


train_data = []
feature_names = ["Budgets", "Review Totals"]
for i in range(len(x_budget_train)):
    train_data.append([x_budget_train[i], x_total_train[i]])
    
test_data = []
for i in range(len(x_budget_test)):
    test_data.append([x_budget_test[i], x_total_test[i]])
df_train = pd.DataFrame(train_data, columns = feature_names)
df_test = pd.DataFrame(test_data, columns = feature_names)
target_train = pd.DataFrame(y_bo_train, columns = ["Box Office"])
target_test = pd.DataFrame(y_bo_test, columns=["Box Office"])


X_train = df_train[feature_names]
X_test = df_test[feature_names]
y_train = target_train["Box Office"]
y_test = target_test["Box Office"]


from sklearn import linear_model
clf = linear_model.LinearRegression().fit(X_train, y_train)
predicted_LR = clf.predict(X_test)

mse = np.mean((predicted_LR - y_test)**2)
variance = r2_score(y_test, predicted_LR)

print("Linear Regression: ")
print("MSE= " + str(mse))
print("variance= " + str(variance))

from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 20, max_depth = 7, random_state = 0).fit(X_train, y_train)
predicted_RF = clf.predict(X_test)

# clf = sm.OLS(y_train, X_train).fit()
# predicted_OLS = clf.predict(X_test)

mse = np.mean((predicted_RF - y_test)**2)
variance = r2_score(y_test, predicted_RF)

print("Random Forest: ")
print("MSE= " + str(mse))
print("variance= " + str(variance))


In [ ]:
print(y_test[0:5])
print(predicted_LR[0:5])
print(predicted_RF[0:5])

In [ ]:
plt.scatter(x_total_train, y_bo_train, color="blue", )
plt.plot(np.unique(x_total_test), 
         np.poly1d(np.polyfit(x_total_test, y_bo_test, 1))(np.unique(x_total_test)), color='red')


plt.xticks(())
plt.yticks(())
plt.title("Linear Regression for Review Counts & Box Office Revenue")
plt.xlabel("Review Counts")
plt.ylabel("Box Office Revenue")
plt.show()